## 03 DATA TRANSFORMATION

In [1]:
import os
from dataclasses import dataclass

Get in the correct working directory

In [2]:
%pwd

'/Users/chrissunderland/Desktop/starbucks_store_predictions/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/chrissunderland/Desktop/starbucks_store_predictions'

In [5]:
import pandas as pd
from src.StarbucksProject.utils.common import read_yaml
from src.StarbucksProject.constants import *

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Create the entity

In [7]:
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path  
    data_path: Path

### Create configuration manager

In [8]:
from src.StarbucksProject.constants import *
from src.StarbucksProject.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:

    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,  
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
    
        config = self.config.data_transformation
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(root_dir=config.root_dir,
                                                              data_path = config.data_path)
        
        return data_transformation_config

### Create the component

In [10]:
from src.StarbucksProject import logger
from sklearn.model_selection import train_test_split

In [19]:
class DataTransformation:

    def __init__(self, config: DataTransformationConfig):

        self.config = config

    def split_data(self):
    
        data = pd.read_csv(self.config.data_path)
        
        X = data.iloc[:, :-1]
        y = data.iloc[:, -1]  # I guess I could have done .target from the config.yaml file
                        
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
        
        logger.info("split data into training and rest sets")
        X_train.to_csv(os.path.join(self.config.root_dir, "X_train.csv"), index=False)
        X_test.to_csv(os.path.join(self.config.root_dir, "X_test.csv"), index=False)
        y_train.to_csv(os.path.join(self.config.root_dir, "y_train.csv"), index=False)
        y_test.to_csv(os.path.join(self.config.root_dir, "y_test.csv"), index=False)
        
        logger.info(X_train.shape)
        logger.info(y_train.shape)
        logger.info(X_test.shape)
        logger.info(y_test.shape)
  

### Create the pipeline

In [18]:
try:

    config = ConfigurationManager()

    data_transformation_config = config.get_data_transformation_config()

    data_transformation = DataTransformation(config=data_transformation_config)
    
    data_transformation.split_data()

    
except Exception as e:

    raise e

[2024-07-11 18:40:04,406: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-11 18:40:04,408: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-11 18:40:04,411: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-11 18:40:04,412: INFO: common: created directory at: artifacts]
[2024-07-11 18:40:04,412: INFO: common: created directory at: artifacts/data_transformation]
[2024-07-11 18:40:04,423: INFO: 3707799590: split data into training and rest sets]
[2024-07-11 18:40:04,430: INFO: 3707799590: (123, 10)]
[2024-07-11 18:40:04,431: INFO: 3707799590: (123,)]
[2024-07-11 18:40:04,431: INFO: 3707799590: (42, 10)]
[2024-07-11 18:40:04,432: INFO: 3707799590: (42,)]


### Modularize the code (once working)

- Return to project folder